## Problem Statement:

The Taxi and Limousine Commission (TLC) of New York City collects trip record data from licensed taxis and for-hire vehicles (FHVs) and provides it to the public. The data includes details such as pick-up and drop-off times, locations, passenger counts, and payment information for each trip. As a data engineer, your task is to build a batch data processing pipeline using PySpark to process and analyze this data to gain insights into taxi and FHV trips in New York City.

### Goals:

Data ingestion: Download the trip record data from the NYC TLC website and ingest it into the pipeline for further processing.

Data cleaning and validation: Perform data quality checks and validation to ensure that the data is clean and consistent. Identify and remove duplicates, null values, and other data quality issues that may impact downstream analysis.

Data transformation: Transform the raw trip record data into a format that is optimized for analysis. This may include aggregating the data by time periods, geographical regions, and other factors of interest.

Data analysis: Use PySpark to perform statistical analysis, data exploration, and data visualization to gain insights into taxi and FHV trips in New York City. This may include identifying popular pick-up and drop-off locations, peak trip times, and other patterns and trends in the data.

Data storage: Store the processed and analyzed data in a suitable data storage system such as Hadoop Distributed File System (HDFS) or Apache Cassandra for future use.

Automation and scheduling: Automate the data processing pipeline using tools such as Apache Airflow or Apache Oozie. Schedule the pipeline to run at regular intervals to ensure that the data is up to date and accurate.

---

The overall goal of the project is to build a batch data processing pipeline using PySpark to extract insights from the NYC TLC trip record data. The pipeline should be scalable, efficient, and automated to enable easy data processing and analysis.

### Import Libraries and Intiate Spark session

In [1]:
import configparser

import pandas as pd

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

In [2]:
spark = SparkSession.builder.appName("nyc_batch_pipeline").getOrCreate()

23/05/14 17:06:06 WARN Utils: Your hostname, joker021-VirtualBox resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/05/14 17:06:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/14 17:06:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

### Data Ingestion

In [4]:
# Parsing from config file
conf = configparser.ConfigParser()
conf.read("config")
data_source_path = conf.get("DATASOURCE PATH", "PATH")

# Reading the DataSource from PySpark
df = spark.read.parquet(data_source_path)

In [5]:
print(f"No of Partitons: {df.rdd.getNumPartitions()}")

No of Partitons: 4


In [6]:
# Schema
df.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: string (nullable = true)
 |-- Trip_Dropoff_DateTime: string (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Start_Lon: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Rate_Code: double (nullable = true)
 |-- store_and_forward: double (nullable = true)
 |-- End_Lon: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)



### Data Cleaning And Validation

#### Missing Values

In [7]:
no_of_row = df.count()
print(f"No of Rows: {no_of_row}")
print(f"No of cols: {len(df.columns)}")

No of Rows: 14092413
No of cols: 18


In [8]:
# Get No of Null vlaues per columns
null_count = df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns])
null_df = null_count.pandas_api().transpose()

/home/joker021/.local/lib/python3.10/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [9]:
null_col_list = null_df[null_df[0] > 0].index.to_list()

/home/joker021/.local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [10]:
# We could see below three cols have huge no of Null Values
# The Rate_Code and mta_tax is completely null
# store_and_forward have few rows present
null_col_list = ["Rate_Code", "store_and_forward", "mta_tax"]
df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in null_col_list]).show()

+---------+-----------------+--------+
|Rate_Code|store_and_forward| mta_tax|
+---------+-----------------+--------+
| 14092413|         14091189|14092413|
+---------+-----------------+--------+



In [13]:
# Checking no of Distinct values in Null having columns
df.select([F.countDistinct(F.col(c)).alias(c) for c in null_col_list]).show()

+---------+-----------------+-------+
|Rate_Code|store_and_forward|mta_tax|
+---------+-----------------+-------+
|        0|                2|      0|
+---------+-----------------+-------+



In [14]:
# Count of distinct values on store_and_forward, we could see the values are very small compared to total rows
df.select(F.col("store_and_forward")).groupBy('store_and_forward').count().show()

+-----------------+--------+
|store_and_forward|   count|
+-----------------+--------+
|              0.0|    1212|
|             null|14091189|
|              1.0|      12|
+-----------------+--------+



In [15]:
# Since the amount of null values is very large compared to total no of records we are dropping those columns
df_not_null = df.drop(*null_col_list)

In [16]:
df_not_null.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: string (nullable = true)
 |-- Trip_Dropoff_DateTime: string (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Start_Lon: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- End_Lon: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)



#### DateTime

In [17]:
# Currently the DateTime col are in String format, will check the format
date_col = ["Trip_Pickup_DateTime", "Trip_Dropoff_DateTime"]
df_not_null.select(date_col).show(3)

+--------------------+---------------------+
|Trip_Pickup_DateTime|Trip_Dropoff_DateTime|
+--------------------+---------------------+
| 2009-01-04 02:52:00|  2009-01-04 03:02:00|
| 2009-01-04 03:31:00|  2009-01-04 03:38:00|
| 2009-01-03 15:43:00|  2009-01-03 15:57:00|
+--------------------+---------------------+
only showing top 3 rows



In [20]:
# We could see the format is %Y-%M-%D %H:%M:%s
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
date_dict_map = {date_c: F.to_timestamp(F.col(date_c)) for date_c in date_col}
df_date_parsed = df_not_null.withColumns(date_dict_map)

In [21]:
df_date_parsed.printSchema()

root
 |-- vendor_name: string (nullable = true)
 |-- Trip_Pickup_DateTime: timestamp (nullable = true)
 |-- Trip_Dropoff_DateTime: timestamp (nullable = true)
 |-- Passenger_Count: long (nullable = true)
 |-- Trip_Distance: double (nullable = true)
 |-- Start_Lon: double (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- End_Lon: double (nullable = true)
 |-- End_Lat: double (nullable = true)
 |-- Payment_Type: string (nullable = true)
 |-- Fare_Amt: double (nullable = true)
 |-- surcharge: double (nullable = true)
 |-- Tip_Amt: double (nullable = true)
 |-- Tolls_Amt: double (nullable = true)
 |-- Total_Amt: double (nullable = true)



#### Distinct Values

In [22]:
string_cols = [f.name for f in df_date_parsed.schema.fields if isinstance(f.dataType, F.StringType)]
distinct_count = df_date_parsed.select([F.countDistinct(F.col(c)).alias(c) for c in string_cols])
distinct_count_pd = distinct_count.pandas_api().transpose()
dist_cols = distinct_count_pd[distinct_count_pd[0]<50].index.tolist()

/home/joker021/.local/lib/python3.10/site-packages/pyspark/pandas/utils.py:975: PandasAPIOnSparkAdviceWarning: `to_list` loads all data into the driver's memory. It should only be used if the resulting list is expected to be small.
  warnings.warn(message, PandasAPIOnSparkAdviceWarning)


In [23]:
df_date_parsed.select([F.countDistinct(F.col(c)).alias(c) for c in dist_cols]).show()

+-----------+------------+
|vendor_name|Payment_Type|
+-----------+------------+
|          3|           6|
+-----------+------------+



In [24]:
# View Distinct Values
for c in dist_cols:
    df_date_parsed.select(c).distinct().show()

+-----------+
|vendor_name|
+-----------+
|        CMT|
|        VTS|
|        DDS|
+-----------+



+------------+
|Payment_Type|
+------------+
|   No Charge|
|        CASH|
|      Credit|
|        Cash|
|     Dispute|
|      CREDIT|
+------------+



#### Duplicates

In [25]:
#check for duplicate values
df_dup_drop = df_date_parsed.dropDuplicates()

In [26]:
# Taking count after duplicates
cnt_after_drop = df_dup_drop.count()

# No of Duplciates dropped
no_of_row - cnt_after_drop

23/05/14 17:09:34 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:34 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:34 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:34 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:34 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:34 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:35 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:35 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:35 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:35 WARN TaskMemoryManager: Failed to allocate a page (4194288 bytes), try again.
23/05/14 17:09:35 WARN TaskMemoryManager

31

### Data Validation